In [1]:
# pip install watermark
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,sklearn -g 
#,statsmodels

CPython 3.6.7
IPython 7.4.0

numpy 1.16.2
scipy 1.2.1
pandas 0.24.2
matplotlib 3.0.3
sklearn 0.20.3

compiler   : GCC 8.2.0
system     : Linux
release    : 4.9.93-boot2docker
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   :


In [2]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
#pip install tqdm
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import json
import math
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
%matplotlib inline
import datetime
from datetime import datetime, date, time,timedelta
import re

In [ ]:
PATH_TO_DATA = '/notebooks/pinda/data'
PATH_TO_STRUCG = '/notebooks/pinda/data/alcanc26_03_2019'
#FILE_NAME_DATA1 = 'TX03.csv'
FILE_NAME_DATA1 = 'test.csv'
IGNORE_CHART = ['@SIMD']



In [33]:
data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_DATA1), encoding="cp1251",sep=';',header=None)
#assign columns
data_a1 = data_a1[[0,1,3,5,11,28,29]]
data_a1.columns = ['rack_cpu','chart','block','signal','value','type','out']
data_a1 = pd.concat([data_a1, data_a1['rack_cpu'].str.split("\\",expand = True)[[0,1]]], axis=1) 
data_a1.columns = ['rack_cpu','chart','block','signal','value','type','out','rack','cpu']
data_a1["out"]= data_a1["out"].replace(("IN","OUT"),(0,1)).astype('bool') 
data_a1 = data_a1[~data_a1['chart'].isin(IGNORE_CHART)]
print(data_a1['type'].unique())
print(data_a1['chart'].unique())

['GLOBAL' 'BOOL' 'STRING' 'INT' 'WORD' 'REAL' 'SDTIME' 'DINT' 'BYTE'
 'DWORD']
['@SND_1' 'ADDT_1' 'BODY_1' 'CCM_01' 'CFPU_1' 'CIRC_1' 'COIL_1' 'COST_1'
 'DRCIR1' 'EMUL_1' 'FHOFF1' 'FSCHN1' 'FTECH1' 'FUME_1' 'GESIG1' 'HEATR1'
 'HPAGC1' 'HPRBB1' 'INHW11' 'LPEXT1' 'LPMIL1' 'MESY11' 'MESY21' 'MESY31'
 'MODU_1' 'MPUM_1' 'MSRC_1' 'OMSB11' 'OMSB21' 'OMSB31' 'OMSB41' 'OMSR_1'
 'OUHW11' 'PERM_1' 'PRFI_1' 'PROI_1' 'RACK_1' 'RCMS_1' 'SEPT_1' 'SERV_1'
 'SKIM_1' 'SPRAY1' 'TPUM_1' 'UTLT_1' 'VENT_1' 'VISU11' 'VISU_1' 'WPS__1']


In [80]:
def get_some_value(rack,chart,block,signal,signal_type,PATH_TO_STRUCG=PATH_TO_STRUCG): 
    try:
        with open(os.path.join(PATH_TO_STRUCG,rack.lower(),chart.lower()+'.ofp')) as f:
            lines = [line.rstrip('\n') for line in f]
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        #print('ERROR File not exists!')
        return -4
    b = re.compile(r"\d+ ?%s +:"%(block))
    sf = re.compile(r"^ *\d+\s+%s\s+"%(signal))
    s = re.compile(r'(([<-] ?)|(@TYP=..,))([^@"\',&]+)([,"\'&]|$)')
    s_string = re.compile(r'(([-])|(@TYP=..,)) +(\'.+\')([,"\&]|$)')
    el = re.compile(r'^ *\d+\s+[+]')
    mark_search_signal = False
    mark_block_found = False
    mark_signal_found = False
    for line in lines:
        if block in line:
            if b.search(line):
                mark_search_signal = True
                mark_block_found = True
                #print("Block found >>> ", line)
                continue
        else:
            if mark_search_signal:
                n = sf.search(line)
                if n:
                    mark_search_signal = False
                    #print(n.group())
                    #print("Signal found >>> ", line)
                    if signal_type == "STRING":
                        m=s_string.search(line)
                    else:
                        m=s.search(line)
                    #print(m)
                    print(m.group(4))
                    if m:
                        #return normal result
                        return m.group(4)
                    else:
                        #print('ERROR Signal can not be recognized!')
                        return -3
                elif el.search(line):
                    return -3
                    
    if not mark_block_found:
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!ERROR No block found!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return -1
    if mark_search_signal:
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!ERROR No signal found!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return -2
    return -1
#@TYP=V2
#1219	X1  N2 < @TYP=O2,5,'Manual'
#value = get_some_value("TX03","EMUL_1","YS__03","X0")
#print("Comparison:      ",j['value'], ">>>",value)

In [32]:
tst_line = "1569	S   B1 < AND_01.Q"
#tst_line = "1433	X2  N2 < @TYP=O2,3,'Warning'"
tst_line = "1592	I5  B1 < 0"
tst_line = "1614	I1  B1 < PUMP_1.E11&"
tst_line = "1619	I1  B1 < LAMP1.Q,'HFMDONLP'&"
tst_line = " 692	I1  B1 < OP4704.Q11,'Exhst_Of'&"
tst_line = "3156	SV  V2 < 0H0000"
tst_line = "2386	IBH V1 < 0H0,!ET09MC.0021"
tst_line = "2425	T   TF <  5           [l]"
tst_line = "3682	X   N4 < 0%,!FI17MC.0001"
tst_line = " 369	X1  NF < 81.54414290%,'CAN_BODY'"
tst_line = " 343	B6  NF < #1.0000000E3,'LITROS'"
s = re.compile(r'((< ?)|(@TYP=..,))([^@"\',&]+)([,"\'&]|$)')
m=s.search(tst_line)
print(m)
print(m.group(4))

<_sre.SRE_Match object; span=(12, 27), match='< #1.0000000E3,'>
#1.0000000E3


In [92]:
tst_line = " 738	T1  NS - '@I@ HIGH PRESS. AGC. SYSTEM READY          '"
s = re.compile(r'(([-])|(@TYP=..,)) +(\'.+\')([,"\&]|$)')
m=s.search(tst_line)
print(m)
print(m.group(4))
val_sim = "5.0E2 [ms]"
units = 1000
tmp_txt = re.sub('[^0-9.E-]','', val_sim)
v_s = int(math.floor(float(tmp_txt)*units))
print(v_s)

<_sre.SRE_Match object; span=(12, 59), match="- '@I@ HIGH PRESS. AGC. SYSTEM READY          '">
'@I@ HIGH PRESS. AGC. SYSTEM READY          '
500000


In [93]:
def toDeci(str,base): 
    def val(c): 
        if c >= '0' and c <= '9': 
            return ord(c) - ord('0') 
        else: 
            return ord(c) - ord('A') + 10; 
    llen = len(str) 
    power = 1 #Initialize power of base 
    num = 0     #Initialize result 
  
    # Decimal equivalent is str[len-1]*1 +  
    # str[len-1]*base + str[len-1]*(base^2) + ...  
    for i in range(llen - 1, -1, -1): 
          
        # A digit in input number must  
        # be less than number's base  
        if val(str[i]) >= base: 
            print('Invalid Number') 
            return -1
        num += val(str[i]) * power 
        power = power * base 
    return num 

def compare_values(val_tdc,val_sim,val_type):
    #['BOOL' 'INT' 'WORD' 'SDTIME' 'GLOBAL' 'STRING' 'REAL' 'DINT' 'BYTE' 'DWORD']
    #convert values first
    con_check = re.compile(r'\S+[.]\D+')
    if con_check.search(val_sim):
        #print("Signal value of simadyn has chart connection")
        return -1
    if val_type=='REAL':
        v_t = float(val_tdc.replace(',','.'))
        if '%' in val_sim:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = float(tmp_txt)/100.0
        else:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = float(tmp_txt)
    elif val_type=='BOOL':
        tmp_txt = val_sim.strip()
        if re.match(r'[10]$', tmp_txt):
            v_s = bool(val_sim)
        v_t = bool(val_tdc)
        
    elif val_type=='SDTIME': 
        v_t = int(val_tdc.strip('ms'))
        units = 1000
        un_txt = val_sim.lower()
        un_txt = un_txt.split("[")[1]
        un_txt = un_txt.split("]")[0]
        if un_txt=="s":
            units = 1000
        elif un_txt=="min":
            units = 60000
        elif un_txt=="ms":
            units = 1
        tmp_txt = re.sub('[^0-9.E-]','', val_sim)
        v_s = int(math.floor(float(tmp_txt)*units))
    elif val_type=='INT':
        #print(val_sim)
        v_t = int(val_tdc)
        fl_check = re.compile(r'\d+[.]\d+')
        if fl_check.search(val_sim):
            #print("Signal in Simadyn has REAL instead INT")
            return -2
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            v_s = toDeci(tmp_txt.strip(),16) 
        elif '%' in val_sim:
            tmp_txt = val_sim.replace('%','')
            tmp_txt = re.sub('[^0-9.E-]','', tmp_txt)
            v_s = int(tmp_txt)
        else:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = int(val_sim)

    elif val_type=='WORD':
        tmp_txt = val_tdc.split('16#')[1]
        v_t = toDeci(tmp_txt.strip(),16)
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            v_s = toDeci(tmp_txt.strip(),16) 
        elif "B" in val_sim:
            tmp_txt = val_sim.split('0B')[1]
            tmp_txt =  tmp_txt.replace(" ", "")
            v_s = toDeci(tmp_txt.strip(),2)
    elif val_type=='BYTE':
        tmp_txt = val_tdc.split('16#')[1]
        v_t = toDeci(tmp_txt.strip(),16)
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            v_s = toDeci(tmp_txt.strip(),16) 
        elif "B" in val_sim:
            tmp_txt = val_sim.split('0B')[1]
            tmp_txt =  tmp_txt.replace(" ", "")
            v_s = toDeci(tmp_txt.strip(),2)
    elif val_type=='STRING':
        v_t = val_tdc.strip("'")
        v_s = val_sim.strip("'")
    #print("TDC: ",v_t)
    #print("SIM: ",v_s)
    return v_t==v_s




In [84]:
data_a1_1 = data_a1
for i, j in data_a1_1.iterrows():
    if(not j['out']) and not pd.isnull(j['value']):
        #print(j['rack'],j['chart'],j['block'],j['signal'])
        value = get_some_value(j['rack'],j['chart'],j['block'],j['signal'],j['type'])
        if value == -1:
            print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!")
        elif value == -2:
            print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!Signal is not found in SimadynD software!!!!!!!!!!")
        elif value == -3:
            print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!Signal can not be recognized!!!!!!!!!!")
        elif value == -4:
            print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!File not exist!!!!!!!!!!")
        else:
            #try:
            compare_result = compare_values(j['value'],value,j['type'])
            if compare_result==-1:
                print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL missing connection    XXXXXXXXXXXXXXXXXXXXXXXXXXX")
            elif compare_result==-2:
                print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX")
            else:
                print(j['rack'],j['chart'],j['block'],j['signal'], "Comparison:",j['value'], ">>>",value,">>>",compare_result)  
                
            #except ValueError:
            #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably float in INT field XXXXXXXXXXXXXXXXXXXXXXXXXXX")
            #except UnboundLocalError:
            #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably lost chart connection XXXXXXXXXXXXXXXXXXXXXXXX")   
        

0
TX03 @SND_1 CMM CDV Comparison: 0 >>> 0 >>> True
TX03 @SND_1 CP52_D21 N01 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N02 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N03 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N04 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N05 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N06 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N07 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N08 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N09 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N10 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 @SND_1 CP52_D21 N11 !!!!!!!!!!ERRO

0                      
TX03 ADDT_1 ADCM45 RCM Comparison: 0 >>> 0                       >>> True
1
TX03 ADDT_1 ADCM45 SEL Comparison: 1 >>> 1 >>> True
0                      
TX03 ADDT_1 ADCM45 FLD Comparison: 0 >>> 0                       >>> True
0                      
TX03 ADDT_1 ADCM46 SER Comparison: 0 >>> 0                       >>> True
0
TX03 ADDT_1 ADCM46 SEL Comparison: 0 >>> 0 >>> True
0                      
TX03 ADDT_1 ADCM46 FB2 Comparison: 0 >>> 0                       >>> True
1                      
TX03 ADDT_1 ADCM46 DIR Comparison: 1 >>> 1                       >>> True
1
TX03 ADDT_1 ADCM46 MCC Comparison: 1 >>> 1 >>> True
1                      
TX03 ADDT_1 ADCM46 PLC Comparison: 1 >>> 1                       >>> True
#3600
TX03 ADDT_1 ADCM47 X2 Comparison: 3600 >>> #3600 >>> True
1
TX03 ADDT_1 ADCM61 EN Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 ADCM61 X1 Comparison: 1 >>> 1 >>> True
3
TX03 ADDT_1 ADCM61 X2 Comparison: 3 >>> 3 >>> True
4
TX03 ADDT_1 ADCM61 X3 Com

TX03 ADDT_1 FREE32 I !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE33 I !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE34 S !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE34 R !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE35 S !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE35 R !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE36 S !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE36 R !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE37 I1 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE37 I2 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE38 I1 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX03 ADDT_1 FREE38 I2 !!!!!!!!!!ERROR Bl

1
TX03 ADDT_1 P1RC06 I4 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1RC07 I2 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1RC07 I3 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1RC07 I4 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1RC08 I2 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1RC08 I3 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1RC08 I4 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR03 I1 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR03 I3 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR03 I4 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR04 I1 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR04 I3 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR04 I4 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR06 I2 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR06 I3 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR06 I4 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR07 I1 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR07 I3 Comparison: 1 >>> 1 >>> True
1
TX03 ADDT_1 P1SR07 I4 Comparison: 1 >>> 1 >>

0
TX03 BODY_1 BDCADD I8 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCADD I9 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCADD I10 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCADD I11 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCADD I12 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCADD I13 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCADD I14 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCADD I15 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCADD I16 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I8 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I9 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I10 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I11 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I12 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I13 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I14 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I15 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM01 I16 Comparison: 0 >>> 0 >>> True
0
TX03 BODY_1 BDCM02 I2 Comparis

...

TX03 OMSB41 MSK111 OR Comparison: 16#0000 >>> 0H0000                  >>> True
0H0000                 
TX03 OMSB41 MSK111 X Comparison: 16#0000 >>> 0H0000                  >>> True
0                      
TX03 OMSB41 MSK111 I Comparison: 0 >>> 0                       >>> True
3[s]
TX03 OMSB41 MTMP01 T Comparison: 3000ms >>> 3[s] >>> True
1
TX03 OMSB41 O1CM01 I2 Comparison: 1 >>> 1 >>> True
0
TX03 OMSB41 O1CM22 I9 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1CM22 I10 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1CM22 I11 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1CM22 I12 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1CM22 I13 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1CM22 I14 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1CM22 I15 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1CM22 I16 Comparison: 0 >>> 0 >>> True
1
TX03 OMSB41 O1CM90 I3 Comparison: 1 >>> 1 >>> True
0
TX03 OMSB41 O1CM90 I4 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1CM90 I5 Comparison: 0 >>> 0 >>> True
0
TX03 OMSB41 O1C

0
TX03 OMSR_1 ALAR03 I3 Comparison: 0 >>> 0 >>> True
0
TX03 OMSR_1 ALAR03 I4 Comparison: 0 >>> 0 >>> True
20[s]
TX03 OMSR_1 ALAR04 T Comparison: 20000ms >>> 20[s] >>> True
1
TX03 OMSR_1 ALAR06 I3 Comparison: 1 >>> 1 >>> True
0
TX03 OMSR_1 ALAR06 I4 Comparison: 0 >>> 0 >>> True
0
TX03 OMSR_1 ALAR07 I Comparison: 0 >>> 0 >>> True
1
TX03 OMSR_1 ALAR08 I2 Comparison: 1 >>> 1 >>> True
1
TX03 OMSR_1 ALAR09 I1 Comparison: 1 >>> 1 >>> True
0
TX03 OMSR_1 ALAR10 I2 Comparison: 0 >>> 0 >>> True
F06052.Q15
TX03 OMSR_1 ANDSE2 I1 XXXXXXXXXXXXXXXX ERROR CRITICAL missing connection    XXXXXXXXXXXXXXXXXXXXXXXXXXX
1
TX03 OMSR_1 ANDST1 I3 Comparison: 1 >>> 1 >>> True
1
TX03 OMSR_1 ANDST1 I4 Comparison: 1 >>> 1 >>> True
3[s]
TX03 OMSR_1 AVOFF2 T Comparison: 3000ms >>> 3[s] >>> True
1
TX03 OMSR_1 AVOFF8 I3 Comparison: 1 >>> 1 >>> True
1[s]
TX03 OMSR_1 DELAY1 T Comparison: 1000ms >>> 1[s] >>> True
30[s]
TX03 OMSR_1 DELAY2 T Comparison: 30000ms >>> 30[s] >>> True
15[s]
TX03 OMSR_1 DELAY4 T Comparison: 15000m

0
TX03 OMSR_1 MESY11 I13 Comparison: 0 >>> 0 >>> True
0
TX03 OMSR_1 MESY11 I14 Comparison: 0 >>> 0 >>> True
0
TX03 OMSR_1 MESY11 I15 Comparison: 0 >>> 0 >>> True
0
TX03 OMSR_1 MESY11 I16 Comparison: 0 >>> 0 >>> True
3[s]
TX03 OMSR_1 MPRE01 T Comparison: 3000ms >>> 3[s] >>> True
3[s]
TX03 OMSR_1 MPRE02 T Comparison: 3000ms >>> 3[s] >>> True
0H0000                 
TX03 OMSR_1 MSK002 XOR Comparison: 16#0000 >>> 0H0000                  >>> True
0HFFFF                 
TX03 OMSR_1 MSK002 AND Comparison: 16#FFFF >>> 0HFFFF                  >>> True
0H0000                 
TX03 OMSR_1 MSK002 OR Comparison: 16#0000 >>> 0H0000                  >>> True
0H0000                 
TX03 OMSR_1 MSK002 X Comparison: 16#0000 >>> 0H0000                  >>> True
0                      
TX03 OMSR_1 MSK002 I Comparison: 0 >>> 0                       >>> True
0H0000                 
TX03 OMSR_1 MSK101 XOR Comparison: 16#0000 >>> 0H0000                  >>> True
0HFFFF                 
TX03 OMSR_1 MSK101 AN

0
TX03 OMSR_1 O1FM91 I16 Comparison: 0 >>> 0 >>> True
1
TX03 OMSR_1 O1FNSH I4 Comparison: 1 >>> 1 >>> True
0
TX03 OMSR_1 O1WAR6 I2 Comparison: 0 >>> 0 >>> True
0
TX03 OMSR_1 O1_FLT I4 Comparison: 0 >>> 0 >>> True
7              
TX03 OMSR_1 O1__01 X0 Comparison: 7 >>> 7               >>> True
1
TX03 OMSR_1 O1__01 EN Comparison: 1 >>> 1 >>> True
1
TX03 OMSR_1 O1__01 X1 Comparison: 1 >>> 1 >>> True
3
TX03 OMSR_1 O1__01 X2 Comparison: 3 >>> 3 >>> True
5
TX03 OMSR_1 O1__01 X3 Comparison: 5 >>> 5 >>> True
7
TX03 OMSR_1 O1__01 X4 Comparison: 7 >>> 7 >>> True
1
TX03 OMSR_1 O1__04 EN Comparison: 1 >>> 1 >>> True
0
TX03 OMSR_1 O1__04 I1 Comparison: 0 >>> 0 >>> True
0
TX03 OMSR_1 O1__04 I2 Comparison: 0 >>> 0 >>> True
2
TX03 OMSR_1 O1__04 X1 Comparison: 2 >>> 2 >>> True
3
TX03 OMSR_1 O1__04 X2 Comparison: 3 >>> 3 >>> True
0
TX03 OMSR_1 O1__04 X3 Comparison: 0 >>> 0 >>> True
1
TX03 OMSR_1 O1__04 X4 Comparison: 1 >>> 1 >>> True
0
TX03 OMSR_1 OFFC10 I2 Comparison: 0 >>> 0 >>> True
0
TX03 OMSR_1 OFF

ValueError: could not convert string to float: '#5.0 '